In [1]:
import altair as alt
from vega_datasets import data
# from altair import Chart, X, Y, Color, Text, Axis
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

import geopandas as gpd

In [2]:
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion")

collisions_all_df = pd.read_csv("../data/SDOT_Collisions_Vehicles_20250421.csv")

/var/folders/5d/g9psjp7j7jl8l_mm5697j11c0000gn/T/ipykernel_52613/221243282.py:5: DtypeWarning: Columns (10,12,14,16,23,31) have mixed types. Specify dtype option on import or set low_memory=False.
  collisions_all_df = pd.read_csv("../data/SDOT_Collisions_Vehicles_20250421.csv")


In [3]:
subset_collisions_df = collisions_all_df[["ST_VEH_TYPE_DESC", "ST_TRAFCONTRL_DESC", "ST_VEH_ACT_DESC", "ST_RDSURF_DESC", "Incident Date Time"]]

In [4]:
subset_collisions_df["date"] = pd.to_datetime(
    subset_collisions_df["Incident Date Time"],
    format="%m/%d/%Y %I:%M:%S %p",
    errors='coerce'
)
subset_collisions_df_2019_2024 = subset_collisions_df.where(
    (subset_collisions_df["date"] > "2017-01-01") &
    (subset_collisions_df["date"] < "2025-01-01")
)

/var/folders/5d/g9psjp7j7jl8l_mm5697j11c0000gn/T/ipykernel_52613/2257392421.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_collisions_df["date"] = pd.to_datetime(


In [5]:
subset_collisions_df_2019_2024.dropna(inplace=True)
subset_collisions_df_2019_2024.shape

(98035, 6)

In [6]:
action_desc_df = subset_collisions_df_2019_2024.groupby(["ST_VEH_ACT_DESC"]).size().reset_index(name="count")
top_five_desc = action_desc_df.sort_values("count", ascending=False).ST_VEH_ACT_DESC[:5].values

In [7]:
# Example: adding one row
summed_everything_else = action_desc_df.where(~(action_desc_df["ST_VEH_ACT_DESC"].isin(top_five_desc)))["count"].sum()
new_row = pd.DataFrame([{"ST_VEH_ACT_DESC": "Everything Else", "count": summed_everything_else}])
action_desc_df = pd.concat([action_desc_df, new_row], ignore_index=True)
action_desc_df


,ST_VEH_ACT_DESC,count
0,Backing,2561.0
1,Changing Lanes,3651.0
2,Going Straight Ahead,49818.0
3,Going Wrong Way on Divided Highway,41.0
4,Going Wrong Way on One-Way Street or Road,90.0
5,Going Wrong Way on Ramp,4.0
6,"Illegally Parked, Occupied",64.0
7,"Illegally Parked, Unoccupied",203.0
8,"Legally Parked, Occupied",619.0
9,"Legally Parked, Unoccupied",5308.0


In [8]:
alt.Chart(subset_collisions_df_2019_2024).mark_line().encode(
    x=alt.X(
        "yearmonth(date):T",
        axis=alt.Axis(title=None, labelAngle=-45, labelFontSize=16),
    ),
    y=alt.Y("count():Q"),
    color=alt.Color(
        "ST_VEH_ACT_DESC:N",
        title="ST_VEH_ACT_DESC",
        sort=alt.SortField("ST_VEH_ACT_DESC"),
    ),
).properties(
    width=600,
    height=400,
)

alt.Chart(...)

In [9]:
def vehicle_type_group(row):
    if "Truck" in row["ST_VEH_TYPE_DESC"]:
        return "Truck"
    elif "Bus" in row["ST_VEH_TYPE_DESC"]:
        return "Bus"
    elif "Mo" in row["ST_VEH_TYPE_DESC"]:
        return "Moped/Motorcycle"
    elif "Car" in row["ST_VEH_TYPE_DESC"]:
        return "Car"
    else:
        return "Other"

subset_collisions_df_2019_2024["vehicle_type"] = subset_collisions_df_2019_2024.apply(vehicle_type_group, axis=1)

In [10]:
desc_to_plot = np.append(top_five_desc, "Everything Else")

In [11]:
driving_activity = alt.Chart(
    action_desc_df.where(action_desc_df["ST_VEH_ACT_DESC"].isin(desc_to_plot)).dropna()
).mark_bar().encode(
    x=alt.X(
        "ST_VEH_ACT_DESC:N",
        axis=alt.Axis(title=None, labelAngle=-45, labelFontSize=16),
        sort=alt.SortField("count", order="descending"),
    ),
    y=alt.Y("count:Q", title="Collisions"),
    color=alt.Color(
        "ST_VEH_ACT_DESC:N",
        title="ST_VEH_ACT_DESC",
        # sort=alt.SortField("ST_VEH_ACT_DESC"),
        scale=alt.Scale(domain=desc_to_plot, range=["#EA1013", '#fc8d62', '#8da0cb', '#e78ac3', '#ffd92f', '#b3b3b3']),
    ).legend(None),
).properties(
    width=200,
    height=400,
)

In [12]:
# mean_gsf = alt.Chart().mark_rule().encode(
#     y=alt.Y(
#         "mean(ST_VEH_ACT_DESC):Q",
#         title="Mean",
#     )
#     color=alt.value('black'),
#     size=alt.value(10),
# )
# mean_gsf

In [13]:
subset_collisions_df_2019_2024.where(
    (subset_collisions_df_2019_2024["ST_TRAFCONTRL_DESC"] == "No Traffic Control") |
    (subset_collisions_df_2019_2024["ST_TRAFCONTRL_DESC"] == "Signals")
).dropna()

,ST_VEH_TYPE_DESC,ST_TRAFCONTRL_DESC,ST_VEH_ACT_DESC,ST_RDSURF_DESC,Incident Date Time,date,vehicle_type
0,Passenger Car,No Traffic Control,Going Straight Ahead,Concrete,8/17/2024 3:16:00 AM,2024-08-17 03:16:00,Car
1,"Pickup, Panel Truck or Vannette Under 10,000 lbs",No Traffic Control,Going Straight Ahead,Blacktop,12/5/2024 2:13:00 AM,2024-12-05 02:13:00,Truck
5,Passenger Car,Signals,Going Straight Ahead,Concrete,12/25/2024 7:21:00 PM,2024-12-25 19:21:00,Car
6,Passenger Car,Signals,Stopped at Signal or Stop Sign,Concrete,12/25/2024 7:21:00 PM,2024-12-25 19:21:00,Car
7,"Pickup, Panel Truck or Vannette Under 10,000 lbs",Signals,Going Straight Ahead,Blacktop,9/11/2024 11:30:00 AM,2024-09-11 11:30:00,Truck
...,...,...,...,...,...,...,...
421159,Passenger Car,No Traffic Control,Changing Lanes,Concrete,6/23/2024 11:35:00 AM,2024-06-23 11:35:00,Car
421160,Passenger Car,No Traffic Control,Going Straight Ahead,Concrete,6/23/2024 11:35:00 AM,2024-06-23 11:35:00,Car
421161,Passenger Car,No Traffic Control,Changing Lanes,Blacktop,7/5/2024 7:47:00 AM,2024-07-05 07:47:00,Car
421162,"Pickup, Panel Truck or Vannette Under 10,000 lbs",No Traffic Control,Going Straight Ahead,Blacktop,7/5/2024 7:47:00 AM,2024-07-05 07:47:00,Truck


In [14]:
# collisions_are_down = alt.Chart(
#     subset_collisions_df_2019_2024.where(
#         (subset_collisions_df_2019_2024["ST_TRAFCONTRL_DESC"] == "No Traffic Control") |
#         (subset_collisions_df_2019_2024["ST_TRAFCONTRL_DESC"] == "Signals")
#     ).dropna(),
#     title=alt.Title(
#         "Seattlites are becoming better drivers!",
#         fontSize=30,
#         align="left",
#         anchor="start",
#         subtitle=[
#             "Seattle vehicle collisions are down by more than 50% since January of 2017. This data is made avaialable",
#             "via the Seattle Open Data portal data.seattle.gov, specifically the SDOT Collisions Vehicles dataset."
#         ],
#         subtitleFontSize=14,
#     )
# ).mark_line(size=15).encode(
#     x=alt.X(
#         "year(date):T",
#         axis=alt.Axis(title=None, labelAngle=-45, labelFontSize=16),
#     ),
#     y=alt.Y("count():Q", title="Collisions"),
#     color=alt.value("green")
# ).properties(
#     width=600,
#     height=400,
# )
# arrow = alt.Chart(pd.DataFrame(
#     {"date": [pd.to_datetime("2024-01-01")], "count": [6300]}
# )).mark_point(shape="triangle", filled=True, fillOpacity=1).encode(
#     # x=alt.datum(5e24),
#     x=alt.X('year(date):T'),
#     y=alt.Y("count:Q"),
#     angle=alt.AngleValue(-6.5),
#     size=alt.SizeValue(800),
#     color=alt.value("green"),
# )
# (collisions_are_down + arrow).save("a2_deceptive.png", scale_factor=5)

In [15]:
collisions_are_down = alt.Chart(
    subset_collisions_df_2019_2024.dropna(),
    title=alt.Title(
        "Seattlites are becoming better drivers!",
        fontSize=30,
        align="left",
        anchor="start",
        subtitle=[
            "Seattle vehicle collisions are down by more than 50% since January of 2017. This data is made avaialable",
            "via the Seattle Open Data portal data.seattle.gov, specifically the SDOT Collisions Vehicles dataset."
        ],
        subtitleFontSize=14,
    )
).mark_line(size=15).encode(
    x=alt.X(
        "year(date):T",
        axis=alt.Axis(title=None, labelAngle=-45, labelFontSize=16),
    ),
    y=alt.Y("count():Q", title="Collisions"),
    color=alt.value("green")
).properties(
    width=800,
    height=400,
)
arrow = alt.Chart(pd.DataFrame(
    {"date": [pd.to_datetime("2024-01-01")], "count": [7180]}
)).mark_point(shape="triangle", filled=True, fillOpacity=1).encode(
    # x=alt.datum(5e24),
    x=alt.X('year(date):T'),
    y=alt.Y("count:Q"),
    angle=alt.AngleValue(-6.5),
    size=alt.SizeValue(800),
    color=alt.value("green"),
)
(collisions_are_down + arrow).save("a2_deceptive.png", scale_factor=5)
collisions_are_down + arrow

alt.LayerChart(...)

In [16]:
# Manually constructed extracting the COMMUTING TO WORK/WORKERS 16 YEARS AND OVER/WORKED AT HOME data
# from the US Census Bureau specifically the ACS 1-Year Estimates Data Profiles with the exception of
# 2020 where we had to use the 5-Year Estimates
# https://data.census.gov/table/ACSDP1Y2017.DP03?q=Seattle+city,+Washington+Employment&g=160XX00US5363000
census_bureau_data = pd.DataFrame(
    [
        {"date": pd.to_datetime("2017-01-01"), "remote_workers": 30794},
        {"date": pd.to_datetime("2018-01-01"), "remote_workers": 34296},
        {"date": pd.to_datetime("2019-01-01"), "remote_workers": 36349},
        {"date": pd.to_datetime("2020-01-01"), "remote_workers": 62635},
        {"date": pd.to_datetime("2021-01-01"), "remote_workers": 205297},
        {"date": pd.to_datetime("2022-01-01"), "remote_workers": 165175},
        {"date": pd.to_datetime("2023-01-01"), "remote_workers": 132724},
    ]
)
census_bureau_data

,date,remote_workers
0,2017-01-01,30794
1,2018-01-01,34296
2,2019-01-01,36349
3,2020-01-01,62635
4,2021-01-01,205297
5,2022-01-01,165175
6,2023-01-01,132724


In [17]:
collisions_are_down = alt.Chart(
    subset_collisions_df_2019_2024.dropna(),
    title=alt.Title(
        "Remote work in Seattle has lead to a decrease in vehicle collisions",
        fontSize=30,
        align="left",
        anchor="start",
        subtitle=[
            "Collisions are down by more than 50% since January of 2017. While, remote work is higher than prepandemic levels. This data is made",
            "avaialable via the Seattle Open Data portal data.seattle.gov, specifically the SDOT Collisions Vehicles dataset."
        ],
        subtitleFontSize=14,
    )
).mark_line(size=15).encode(
    x=alt.X(
        "year(date):T",
        axis=alt.Axis(title=None, labelAngle=-45, labelFontSize=16),
    ),
    y=alt.Y("count():Q", title="Collisions"),
    color=alt.value("green")
)

collision_arrow = alt.Chart(pd.DataFrame(
    {"date": [pd.to_datetime("2024-01-01")], "count": [7180]}
)).mark_point(shape="triangle", filled=True, fillOpacity=1).encode(
    # x=alt.datum(5e24),
    x=alt.X('year(date):T'),
    y=alt.Y("count:Q"),
    angle=alt.AngleValue(-6.5),
    size=alt.SizeValue(800),
    color=alt.value("green"),
)

remote_work_is_up = alt.Chart(
    census_bureau_data
).mark_line(size=15).encode(
    x=alt.X(
        "year(date):T",
        axis=alt.Axis(title=None, labelAngle=-45, labelFontSize=16),
    ),
    y=alt.Y("remote_workers:Q", title="Number of Remote Workers", axis=alt.Axis(orient="right")),
    color=alt.value("blue")
)

remote_arrow = alt.Chart(pd.DataFrame(
    {"date": [pd.to_datetime("2023-01-01")], "count": [133180]}
)).mark_point(shape="triangle", filled=True, fillOpacity=1).encode(
    # x=alt.datum(5e24),
    x=alt.X('year(date):T'),
    y=alt.Y("count:Q"),
    angle=alt.AngleValue(-6.5),
    size=alt.SizeValue(800),
    color=alt.value("blue"),
)

collision_label = alt.Chart({'values':[{}]}).mark_text(
    align='left',
    dx=5,
    dy=5,
    fontSize=16,
    color='green'
).encode(
    x=alt.value(700),       # x position on the chart
    y=alt.value(250),       # y position on the chart
    text=alt.value('Collisions')  # static text label
)

remote_label = alt.Chart({'values':[{}]}).mark_text(
    align='left',
    dx=5,
    dy=5,
    fontSize=16,
    color='blue'
).encode(
    x=alt.value(650),       # x position on the chart
    y=alt.value(100),       # y position on the chart
    text=alt.value('Remote Work')  # static text label
)

earnest = alt.layer((collisions_are_down + collision_arrow + collision_label), (remote_work_is_up + remote_arrow + remote_label)).resolve_scale(
    y = 'independent'
).properties(
    width=800,
    height=400,
)
earnest.save("a2_earnest.png", scale_factor=5)
earnest

alt.LayerChart(...)

#### Analysis of how I made this plot:

##### TODO: Update this explanation

When considering the question of how the number of days of precipitation and the hours of sunshine change throughout the year for 6 major US cities, there were many graphical ways to answer this question. Some key elements I wanted to be sure were present in my visualization include a way to differentiate between cities, a means to determine what month a trend is taking place in, and of course the information about days of precipitation and hours of sunshine. The final design uses small multiples or faceting to convey different charts and their trends per city next to one another. This makes for easy visual comparison from city to city. Originally I was going to create this plot where the faceting was in terms of months and then each subplot contained information about precipitation versus sunshine hours where each point on the chart was a city. This was confusing to compare one city across each month because you had to remember which color it was and look for that color on each plot. In this final design it is relatively simple to glean information answering our question by looking at just one of the plots, and as an added bonus, one can compare this trend to other cities quite easily.

Furthermore, once I decided to facet with respect to the cities, I needed to determine how to present the information in each subplot. I was originally plotting precipitation days versus precipitation inches where each point was a month and sized by the hours of sunshine. This is problematic because it was odd to have a month be a single point losing the temporal ordering of the months. Additionally, this was overly complicated by plotting months, hours of sunshine, precip days, and precip inches with color, size, y-axis and x-axis respectively. I was definitely overplotting. Eliminating the precip inches information all together (after all it didn't contribute to answering the question at hand) freed up the x-axis to put the month information there. This created a nice temporal ordering for the information about days of precipitation.

At this point, I had determined the variable to facet with respect to, I had also determined which variables to plot along the x and y axes. Now I needed to determine if or how I would vary the color and size of the markers as well as any other stylistic choices for the chart. I initially liked conveying the hours of sunshine data by both the size and color shade (using a colormap) but to the viewer it was unclear and confusing to have one piece of information converted in two ways. This led to an overly complicated plot that muddled the quick interpretability of the data.
Therefore, I settled on only varying the color shade by the hours of sunshine and holding the size of the marker constant.

Finally, there were a few key minutia to highlight which improved the overall graphic. One, I added light grey edgecolor to the markers to help them stand out even when the color was fainter due to having more hours of sunshine that particular month. Two, I increased the number of ticks and gridlines for the y-axis to make it easy to compare the values of the points from city to city. Third, I increased the font sizes across the graphic to be appropriate according to their purpose but to always be large and readable. Fourth, In order to allow for more compact comparison, the faceting is in two rows of 3 columns, but to accommodate the compactness of this presentation the month names labeling the x-axis are angled at -45 degrees. Finally, I chose a colormap which conveys the reality that more sunshine hours would be brighter and lighter while less sunshine hours are darker. These last little details help bring the whole graphic together to easily answer the question at hand.
